In [1]:
with open("input.txt") as f:
    lines = [l.strip() for l in f.readlines() if l.strip()]

# part 1

In [2]:
seeds = [int(seed) for seed in lines[0].split(":")[1].split()]

def map_key_and_val(line):
    map_name = line.split()[0]
    k = map_name.split("-to-")[0]
    v = map_name.split("-to-")[1]
    return k, v

maps = {}

for line in lines[1:]:
    if "map" in line:
        cur_key, cur_val = map_key_and_val(line)
        maps[cur_key] = {cur_val: []}
        continue

    dest_start, source_start, range_len = [int(val) for val in line.split()]
    maps[cur_key][cur_val] += [
        {
            "dest_start": dest_start,
            "source_start": source_start,
            "range_len": range_len,
        }
    ]


lowest_loc = float("inf")
map_lines = [map_key_and_val(line) for line in lines if "map" in line]
for seed in seeds:
    val = seed
    for k, v in map_lines:
        for m in maps[k][v]:
            if val >= m["source_start"] and val < m["source_start"]+m["range_len"]:
                val = m["dest_start"] + val - m["source_start"]
                break

    lowest_loc = min(lowest_loc, val)

print("part 1:", lowest_loc)

part 1: 910845529


# part 2

In [3]:
from bisect import bisect_left
import math


line1_vals = [int(v) for v in lines[0].split(":")[1].split()]
seeds = []
for i in range(0, len(line1_vals), 2):
    seed, r = line1_vals[i], line1_vals[i+1]
    seeds += [(seed, r)]


def map_key_and_val(line):
    map_name = line.split()[0]
    k = map_name.split("-to-")[0]
    v = map_name.split("-to-")[1]
    return k, v

maps = {}

for line in lines[1:]:
    if "map" in line:
        cur_key, cur_val = map_key_and_val(line)
        maps[cur_key] = {cur_val: [], "ranges": []}
        continue

    dest_start, source_start, range_len = [int(val) for val in line.split()]
    maps[cur_key][cur_val] += [
        {
            "dest_start": dest_start,
            "source_start": source_start,
            "range_len": range_len,
        }
    ]
    maps[cur_key]["ranges"] += [source_start, source_start+range_len]


for k in maps.keys():
    maps[k]["ranges"] = sorted(maps[k]["ranges"])


def num_can_advance(ranges, val) -> int:
    l = bisect_left(ranges, val)
    if l >= len(ranges):
        return float("inf")

    return ranges[l] - val


lowest_loc = float("inf")
map_lines = [map_key_and_val(line) for line in lines if "map" in line]
for s, r in seeds:
    seed = s
    while seed < s+r:
        skip = float("inf")
        val = seed
        for k, v in map_lines:
            skip = min(skip, num_can_advance(maps[k]["ranges"], val))
            for m in maps[k][v]:
                if val >= m["source_start"] and val < m["source_start"]+m["range_len"]:
                    val = m["dest_start"] + val - m["source_start"]
                    break

        lowest_loc = min(lowest_loc, val)
        if math.isinf(skip):
            break
        elif skip == 0:
            skip = 1

        seed += skip


print("part 2:", lowest_loc)

part 2: 77435348
